In [1]:
# Example SQLite dataset downloaded externally --BONUS for demonstrating system flexibility with undetermined schemas
!python kaggle_database_downloader.py "gastonsaracusti/model-car-mint-classics"

100%|███████████████████████████████████████| 26.0k/26.0k [00:00<00:00, 399kB/s]
Extracting files...

Dataset downloaded to:
"/home/mohamed/.cache/kagglehub/datasets/gastonsaracusti/model-car-mint-classics/versions/2"

Dataset moved to:
"/home/mohamed/Projects/llm-for-sql-queries/data/gastonsaracusti/model-car-mint-classics"

Database File:
"/home/mohamed/Projects/llm-for-sql-queries/data/gastonsaracusti/model-car-mint-classics/data.sqlite"

Process exited with status code: 0


In [2]:
# Database creator through the template given by ASAP Systems for Barcloud
!python asap_database_creator.py "aayman/asap-database-template" "./script.sql"

Database created successfully!

Database File:
/home/mohamed/Projects/llm-for-sql-queries/data/aayman/asap-database-template/data.sqlite


In [3]:
from http.server import HTTPServer, BaseHTTPRequestHandler
import sqlite3
import keyring
from string import Template
import os
import logging
import re
import json
import requests

from utils import Timer, read_task_template
from chatbots import OpenAIChatBot
from constants import OPENAI_API_KEY, MODEL_NAME, DB_FILEPATH

In [4]:
chatbot = OpenAIChatBot(
    api_key    = OPENAI_API_KEY,
    model      = MODEL_NAME,
    db_filepath = DB_FILEPATH,
)

response_details = chatbot.send('What is the total billed amount for the last quarter?')
response_details

{'SQL': "SELECT SUM(TotalAmount) as TotalBilled FROM Bills WHERE BillDate >= date('now', 'start of quarter', '-1 quarter') AND BillDate < date('now', 'start of quarter')",
 'token_usage': {'prompt_tokens': 3835,
  'completion_tokens': 128,
  'total_tokens': 3963},
 'model': 'gpt-4o-mini-2024-07-18',
 'latency_ms': 6537,
 'status': 'ok',
 'Answer': "The query aimed to calculate the total billed amount for the last quarter. However, it returned 'None', indicating that there were no bills recorded in the Bills table for the specified date range. This could suggest either a lack of transactions or that the records for the last quarter have not yet been entered into the database."}

In [ ]:
class Service(BaseHTTPRequestHandler):
    
    def __init__(self, *args, chatbot, **kwargs):
        super().__init__(*args, **kwargs)
        self._chatbot = chatbot
    
    @property
    def chatbot(self):
        return self._chatbot
    
    def do_GET(self):
        pass
    
    def do_POST(self):
        pass
    
    def _send(self, status_code, json_payload):
        try:
            self.send_response(status_code)
            self.send_header('Content-Type', 'application/json; charset=utf-8')
            self.end_headers()
            payload_bytes = json.dumps(json_payload, indent=4).encode('utf-8')
            self.wfile.write(payload_bytes)
        except:
            logging.exception(f'Error sending response.\nKindly check the error logs for traceback details.')
            self.send_error(500, 'Internal server error')
            
    def _receive(self):
        try:
            content_length = int(self.headers.get('Content-Length', 0))
            if content_length == 0:
                return None
            payload_bytes = self.rfile.read(content_length)
            json_payload = json.loads(payload_bytes.decode('utf-8'))
            return json_payload
        except json.JSONDecodeError:
            message = 'Invalid JSON payload.'
            logging.exception(f'{message}\nKindly check the error logs for traceback details.')
            self._send(400, {'error': message})
        except:
            message = 'Error processing request.'
            logging.exception(f'{message}\nKindly check the error logs for traceback details.')
            self._send(400, {'error': message})
        return None

In [8]:
service = BaseHTTPRequestHandler()
service

TypeError: BaseRequestHandler.__init__() missing 3 required positional arguments: 'request', 'client_address', and 'server'

In [6]:
# class HttpRequestHandler(BaseHTTPRequestHandler):
    
#     def __init__(self, chatbot):
#         super().__init__()
#         self._chatbot = chatbot

#     def serve_get(self):
#         if self.path == '/api':
#             self._send({'status': 'Server is running'}, 200)
#         else:
#             self._send({'error': 'Endpoint not found'}, 404)
    
#     def serve_post(self):
#         if self.path == '/api/sql-chat':
#             json_payload = self._receive()
#             response_details = self._chatbot.send(json_payload['message'])
#             status_code = 200 if response_details['status'] == 'ok' else 403
#             self._send(response_details, status_code)
#         else:
#             self._send({'error': 'Endpoint not found'}, 404)
    
#     def _send(self, json_payload, status):
#         self.send_response(status)
#         self.send_header('Content-Type', 'application/json')
#         self.end_headers()
#         self.wfile.write(json.dumps(json_payload, indent=4).encode())
    
#     def _receive(self):
#         content_length = int(self.headers.get('Content-Length', 0))
#         json_payload   = json.loads(
#             self.rfile.read(content_length).decode('utf-8')
#         )
#         return json_payload

# server = HTTPServer(('localhost',8000), HttpRequestHandler)
# server.serve_forever()

In [7]:
# from http.server import HTTPServer, BaseHTTPRequestHandler
# import json
# from urllib.parse import urlparse, parse_qs

# class Server(BaseHTTPRequestHandler):
#     def __init__(self, db_filepath):